# Chapter 8: Memory Management

Key Takeaways:
- **Session Management** allows agents to maintain multi-turn conversations through various backends (`InMemory`, `Database`, `VertexAI`).
- **State Management** provides a "scratchpad" for agents to store and update contextual information during a session.
- **Long-Term Memory** leverages Retrieval Augmented Generation (RAG) to persist and search knowledge across multiple sessions.

### Heuristic: *Context is everything.*

## Setup and Initialization 

In [ ]:
import os
from dotenv import load_dotenv
from google.adk.sessions import InMemorySessionService, DatabaseSessionService, VertexAiSessionService
from google.adk.agents import LlmAgent
from google.adk.agents.invocation_context import InvocationContext
from google.adk.runners import Runner
from google.genai.types import Content, Part
from google.adk.memory import InMemoryMemoryService, VertexAiRagMemoryService
from google.adk.tools.tool_context import ToolContext

load_dotenv()

# --- Configuration ---
REASONING_ENGINE_ID = os.getenv("REASONING_ENGINE_ID")
RAG_ID = os.getenv("RAG_ID")
PROJECT_ID = os.getenv("PROJECT_ID")
LOCATION = os.getenv("LOCATION", "us-central1")

print(f"✅ Configuration Loaded:")
print(f"   Reasoning Engine ID: {REASONING_ENGINE_ID}")
print(f"   RAG ID: {RAG_ID}")

✅ Configuration Loaded:
   Reasoning Engine ID: projects/878018164626/locations/us-central1/reasoningEngines/7485911668015235072
   RAG ID: projects/gen-lang-client-0223183954/locations/us-east1/ragCorpora/4611686018427387904


## Session Management

Session services keep track of each chat conversation. You can choose different backends based on your needs.

### 1. In-Memory Session (Development)

Suitable for local development and testing where persistence is NOT required.

In [2]:
session_service = InMemorySessionService()
print("✅ InMemorySessionService initialized.")

✅ InMemorySessionService initialized.


### 2. Database Session (Persistent)

Suitable for production or development requiring persistent storage across restarts.

In [3]:
# Example using a local SQLite file with aiosqlite for async support:
db_url = "sqlite+aiosqlite:///./my_agent_data.db"
session_service_db = DatabaseSessionService(db_url=db_url)
print(f"✅ DatabaseSessionService initialized with: {db_url}")

✅ DatabaseSessionService initialized with: sqlite+aiosqlite:///./my_agent_data.db


### 3. Vertex AI Session (Production)

Leverages Google Cloud's Vertex AI infrastructure for scalable session management.

In [4]:
if REASONING_ENGINE_ID and PROJECT_ID:
    # The app_name used with this service should correspond to the Reasoning Engine ID
    REASONING_ENGINE_APP_NAME = REASONING_ENGINE_ID
    
    session_service_vca = VertexAiSessionService(project=PROJECT_ID, location=LOCATION)
    print(f"✅ VertexAiSessionService initialized for engine: {REASONING_ENGINE_ID}")
else:
    print("⚠️ REASONING_ENGINE_ID or PROJECT_ID not found in .env. Skipping initialization.")

✅ VertexAiSessionService initialized for engine: projects/878018164626/locations/us-central1/reasoningEngines/7485911668015235072


## State Management

State allows you to store custom data along with the session.

### 1. Simple State: `output_key`

Automatically capture the agent's last response into the session state.

In [5]:
# Define an LlmAgent with an output_key.
greeting_agent = LlmAgent(
    name="Greeter",
    model="gemini-2.5-flash",
    instruction="Generate a short, friendly greeting.",
    output_key="last_greeting"
)

app_name, user_id, session_id = "state_app", "user1", "session1"
session_service = InMemorySessionService()
runner = Runner(
    agent=greeting_agent,
    app_name=app_name,
    session_service=session_service
)

session = await session_service.create_session(
    app_name=app_name,
    user_id=user_id,
    session_id=session_id
)

print(f"Initial state: {session.state}")

# Run the agent
user_message = Content(parts=[Part(text="Hello")])
for event in runner.run(user_id=user_id, session_id=session_id, new_message=user_message):
    if event.is_final_response():
        print("Agent responded.")

# Check updated state - ensure keyword arguments are used
updated_session = await session_service.get_session(app_name=app_name, user_id=user_id, session_id=session_id)
print(f"State after agent run: {updated_session.state}")

Initial state: {}
Agent responded.
State after agent run: {'last_greeting': 'Hi there! Nice to meet you. 😊'}


### 2. Advanced State: Tool-Based Updates

Use tools to update state explicitly during a conversation.

In [6]:
import time

def log_user_login(tool_context: ToolContext) -> dict:
    """Updates the session state upon a user login event."""
    state = tool_context.state
    login_count = state.get("user:login_count", 0) + 1
    state["user:login_count"] = login_count
    state["task_status"] = "active"
    state["user:last_login_ts"] = time.time()
    return {"status": "success", "message": f"Login tracked. Logins: {login_count}"}

# Create an agent that uses the login tool
login_agent = LlmAgent(
    name="LoginTracker",
    model="gemini-2.5-flash",
    instruction="You are a login tracker. When the user says 'login', call the log_user_login tool.",
    tools=[log_user_login]
)

# Setup session with initial state
session_service = InMemorySessionService()
app_name, user_id, session_id = "state_app_tool", "user3", "session3"
session = await session_service.create_session(
    app_name=app_name, 
    user_id=user_id, 
    session_id=session_id,
    state={"user:login_count": 0, "task_status": "idle"}
)

print(f"Initial state: {session.state}")

# Run the agent to trigger the tool
runner = Runner(
    agent=login_agent,
    app_name=app_name,
    session_service=session_service
)

user_message = Content(parts=[Part(text="login")])
for event in runner.run(user_id=user_id, session_id=session_id, new_message=user_message):
    if event.is_final_response():
        print("Agent processed login.")

# Check updated state
updated_session = await session_service.get_session(app_name=app_name, user_id=user_id, session_id=session_id)
print(f"State after tool execution: {updated_session.state}")

Initial state: {'task_status': 'idle', 'user:login_count': 0}


/opt/anaconda3/envs/asd/lib/python3.11/site-packages/google/adk/flows/llm_flows/base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:


Agent processed login.
State after tool execution: {'task_status': 'active', 'user:login_count': 1, 'user:last_login_ts': 1768061129.6008341}


## Long-Term Memory

Memory services provide searchable, persistent knowledge.

### 1. In-Memory Memory (Development)

Lost when the application stops.

In [7]:
memory_service = InMemoryMemoryService()
print("✅ InMemoryMemoryService initialized.")

✅ InMemoryMemoryService initialized.


### 2. Vertex AI RAG Memory (Production)

Leverages Vertex AI RAG for persistent, searchable memory.

In [8]:
if RAG_ID and PROJECT_ID:
    RAG_CORPUS_RESOURCE_NAME = RAG_ID
    
    memory_service_vca = VertexAiRagMemoryService(
        rag_corpus=RAG_CORPUS_RESOURCE_NAME,
        similarity_top_k=5,
        vector_distance_threshold=0.7
    )
    print(f"✅ VertexAiRagMemoryService initialized for corpus: {RAG_ID}")
else:
    print("⚠️ RAG_ID or PROJECT_ID not found in .env. Skipping initialization.")

✅ VertexAiRagMemoryService initialized for corpus: projects/gen-lang-client-0223183954/locations/us-east1/ragCorpora/4611686018427387904


## Conclusion

Memory and State systems allow your agents to "remember" who the user is and what has happened previously. This is crucial for building sophisticated, reliable agentic workflows.